In [1]:
import scipy
import os
import cv2
import pickle
import glob
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import defaultdict
from keras.preprocessing import image
import random

In [4]:
corner_keys = ["Center","Left_up","Left_down","Right_up","Right_down"]

Debug_Print_AUG=False

In [5]:
def save_figures_from_video(dataset_video_path, video_filename, suffix,figures_path,skip_frames = 25,apply_norm = True, apply_diff = True,fix_len = None):
    seq_len = 0

    video_figures_path = os.path.join(figures_path ,video_filename)
    if not os.path.exists(video_figures_path):
        os.makedirs(video_figures_path)

    video_file = os.path.join(dataset_video_path, video_filename + suffix)
    label = 0
    print('Extracting frames from video: ', video_file)

    videoCapture = cv2.VideoCapture(video_file)
    if fix_len is not None:
        vid_len = int(videoCapture.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames = int(float(vid_len)/float(fix_len))
    videoCapture.set(cv2.CAP_PROP_POS_MSEC, (seq_len * skip_frames))
    success, figure_ = videoCapture.read()
    success = True
    files = []
    while success:
        success, figure = videoCapture.read()

        if seq_len % skip_frames == 0:
            if success:
                figure_curr = figure
                image_file = os.path.join(video_figures_path , "frame_%d.jpg" % seq_len)
                files.append(image_file)
                cv2.imwrite(image_file, figure_curr)
        seq_len += 1
    video_images = dict(images_path = video_figures_path, name = video_filename,
                        images_files = files, sequence_length = seq_len, label = label)

    return video_images

In [36]:
# videos_labels = []
def createDataset(dataset_video_path, dataset_name, figure_output_path,fix_len, force = False):
    fix_len=10
    videos_seq_length = []
    datasets_images = {}
    videos_frames_paths = []
    videos_labels = []
        #Extract images for each video for each dataset
    dataset_figures_path = os.path.join(figure_output_path,dataset_name)
    if not os.path.exists(dataset_figures_path):
        os.makedirs(dataset_figures_path)
    dataset_images = []
    for filename in os.listdir(dataset_video_path):
        if filename.endswith(".mp4"):
            video_images_file = os.path.join(dataset_figures_path,filename[:-4], 'video_summary.pkl')
            if os.path.isfile(video_images_file) and not force:
                with open(video_images_file, 'rb') as f:
                    video_images = pickle.load(f)
                    #cv.imshow(video_images)
            else:
                video_images = save_figures_from_video(dataset_video_path, filename[:-4],filename[-4:], dataset_figures_path, fix_len =fix_len)
                if dataset_name == "hocky":
                    if filename.startswith("NV"):
                        video_images['label'] = 0
                    elif filename.startswith("V"):
                        video_images['label'] = 1;
                with open(video_images_file, 'wb') as f:
                    pickle.dump(video_images, f, pickle.HIGHEST_PROTOCOL)
                    dataset_images.append(video_images)
                    videos_seq_length.append(video_images['sequence_length'])
                    videos_frames_paths.append(video_images['images_path'])
                    videos_labels.append(video_images['label'])
            datasets_images[dataset_name] = dataset_images
            
    avg_length = int(float(sum(videos_seq_length)) / max(len(videos_seq_length), 1))

    print(videos_labels)
    
    train_path, test_path, train_y, test_y =  train_test_split(videos_frames_paths,videos_labels, test_size=0.20, random_state=42)
    train_path, valid_path, train_y, valid_y = train_test_split(train_path, train_y, test_size=0.20, random_state=42)
    return train_path,valid_path, test_path, train_y, valid_y, test_y, bin

In [8]:
def frame_loader(frames,figure_shape,to_norm = True):
    output_frames = []
    for frame in frames:
        image = load_img(frame, target_size=(figure_shape, figure_shape),interpolation='bilinear')
        img_arr = img_to_array(image)
        # Scale
        figure = (img_arr / 255.).astype(np.float32)
        # Normalize
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        figure = (figure - mean) / std
        output_frames.append(figure)
    return output_frames

In [9]:
def data_generator(data_paths,labels,batch_size,figure_shape,seq_length,use_aug,use_crop,crop_x_y,classes = 1):
    while True:
        indexes = np.arange(len(data_paths))
        np.random.shuffle(indexes)
        select_indexes = indexes[:batch_size]
        data_paths_batch = [data_paths[i] for i in select_indexes]
        labels_batch = [labels[i] for i in select_indexes]

        X, y = get_sequences(data_paths_batch,labels_batch,figure_shape,seq_length, classes, use_augmentation = use_aug,use_crop=use_crop,crop_x_y=crop_x_y)

        yield X, y

In [11]:
def data_generator_files(data,labels,batch_size):
    while True:
        indexes = np.arange(len(data))
        np.random.shuffle(indexes)
        select_indexes = indexes[:batch_size]
        X = [data[i] for i in select_indexes]
        y = [labels[i] for i in select_indexes]
        yield X, y

In [12]:
def crop_img__remove_Dark(img,x_crop,y_crop,x,y,figure_size):
    x_start = x_crop
    x_end = x-x_crop
    y_start = y_crop
    y_end = y-y_crop
    return cv2.resize(img[y_start:y_end,x_start:x_end,:],(figure_size,figure_size))

In [13]:
def crop_img(img,figure_shape,percentage=0.8,corner="Left_up"):
    if(corner == None):
        corner = random.choice(corner_keys)

    if corner not in corner_keys:
        raise ValueError(
            'Invalid corner method {} specified. Supported '
            'corners are {}'.format(
                corner,
                ", ".join(corner_keys)))

    resize = int(figure_shape*percentage)

    if(corner =="Left_up"):
        x_start = 0
        x_end = resize
        y_start = 0
        y_end = resize
    if (corner == "Right_down"):
        x_start = figure_shape-resize
        x_end = figure_shape
        y_start = figure_shape-resize
        y_end = figure_shape
    if(corner =="Right_up"):
        x_start = 0
        x_end = resize
        y_start = figure_shape-resize
        y_end = figure_shape
    if (corner == "Left_down"):
        x_start = figure_shape-resize
        x_end = figure_shape
        y_start = 0
        y_end = resize
    if (corner == "Center"):
        half = int(figure_shape*(1-percentage))
        x_start = half
        x_end = figure_shape-half
        y_start = half
        y_end = figure_shape-half

    img = cv2.resize(img[y_start:y_end,x_start:x_end, :], (figure_shape, figure_shape)).astype(np.float32)
    return img

In [34]:
def get_sequences(data_paths,labels,figure_shape,seq_length,classes=1, use_augmentation = False,use_crop=True,crop_x_y=None):
    X, y = [], []
    seq_len = 0
    x_=0
    for data_path, label in zip(data_paths,labels):
        frames = sorted(glob.glob(os.path.join(data_path, '*jpg')))
        x = frame_loader(frames, figure_shape)
        if(crop_x_y):
            x = [crop_img__remove_Dark(x_1,crop_x_y[0],crop_x_y[1],x_1.shape[0],x_1.shape[1],figure_shape) for x_1 in x]
        if use_augmentation:
            rand = scipy.random.random()
            corner=""
            if rand > 0.5:
                if(use_crop):
                    corner=random.choice(corner_keys)
                    x = [crop_img(x_2,figure_shape,0.7,corner) for x_2 in x]
                x = [frame.transpose(1, 0, 2) for frame in x]
                if(Debug_Print_AUG):
                    to_write = [list(a) for a in zip(frames, x)]
                    [cv2.imwrite(x_3[0] + "_" + corner, x_3[1] * 255) for x_3 in to_write]

        x = [x[i] - x[i+1] for i in range(len(x)-1)]
        X.append(x)
        y.append(label)
    X = pad_sequences(X, maxlen=seq_length, padding='pre', truncating='pre')
    if classes > 1:
        x_ = to_categorical(x_,classes)
    return np.array(X), np.array(y)

In [15]:
import re

def natural_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

In [16]:
from keras import Input
from keras.callbacks import Callback
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, ConvLSTM2D, Reshape, BatchNormalization, Activation
from tensorflow.keras.layers import LSTM
from keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys
import logging
from keras.applications import Xception, ResNet50, InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback

In [17]:
ResNet50_base_model = ResNet50(weights='imagenet', include_top=False) #include_top= False excludes final fc layer
print('ResNet50 v3 base model without FC loaded')

ResNet50 v3 base model without FC loaded


In [52]:
seq_len=10
size=244
dropout = 0.0
input_layer = Input(shape=(seq_len, size, size, 3))
cnn = TimeDistributed(ResNet50_base_model)(input_layer)
lstm=ConvLSTM2D(256, (3, 3), padding='same', return_sequences=False)(cnn)

lstm = MaxPooling2D(pool_size=(2, 2))(lstm)
flat = Flatten()(lstm)

flat = BatchNormalization()(flat)
flat = Dropout(dropout)(flat)
linear = Dense(1000)(flat)

relu = Activation('relu')(linear)
linear = Dense(256)(relu)
linear = Dropout(dropout)(linear)
relu = Activation('relu')(linear)
linear = Dense(10)(relu)
linear = Dropout(dropout)(linear)
relu = Activation('relu')(linear)

activation = 'sigmoid'
loss_func = 'binary_crossentropy'
classes=1
dropout = 0.0

In [42]:
predictions = Dense(1,  activation=activation)(relu)

model = Model(inputs=input_layer, outputs=predictions)

for layer in ResNet50_base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10, 244, 244, 3)  0         
                             ]                                   
                                                                 
 time_distributed_1 (TimeDis  (None, 10, 8, 8, 2048)   23587712  
 tributed)                                                       
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 8, 8, 256)         21234688  
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 256)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 4096)              0         
                                                           

In [21]:
def get_generators(dataset_name, dataset_videos, datasets_frames, fix_len, figure_size, force, classes=1, use_aug=False,
                   use_crop=True, crop_dark=True):
    train_path, valid_path, test_path, train_y, valid_y, test_y, avg_length = createDataset(dataset_videos, dataset_name, datasets_frames, fix_len, force=force)

    if fix_len is not None:
        avg_length = fix_len
    crop_x_y = None
    if (crop_dark):
        crop_x_y = (11, 38)

    batch_size=2

    len_train, len_valid = len(train_path), len(valid_path)
    train_gen = data_generator(train_path, train_y, batch_size, figure_size, avg_length, use_aug=use_aug, use_crop=use_crop, crop_x_y=crop_x_y, classes=classes)
    validate_gen = data_generator(valid_path, valid_y, batch_size, figure_size, avg_length, use_aug=False, use_crop=False, crop_x_y=crop_x_y, classes=classes)
    test_x, test_y = get_sequences(test_path, test_y, figure_size, avg_length, crop_x_y=crop_x_y, classes=classes)

    return train_gen, validate_gen, test_x, test_y, avg_length, len_train, len_valid

In [67]:
dataset_video_path=r'E:\project-dataset'
figure_output_path=r'E:\project-frames-violence'
dataset_name='hocky'
fix_len=10
figure_size=244
force=True
use_aug=True
classes=1
batch_size=5
batch_epoch_ratio = 0.5
epochs=25
patience_es=15
patience_lr=5
train_gen, validate_gen, test_x, test_y, seq_len, len_train, len_valid = get_generators(dataset_name,dataset_video_path,figure_output_path, fix_len,figure_size,force=force,classes=classes, use_aug=use_aug,use_crop=True,crop_dark=True)

Extracting frames from video:  E:\project-dataset\NV_1.mp4
Extracting frames from video:  E:\project-dataset\NV_10.mp4
Extracting frames from video:  E:\project-dataset\NV_100.mp4
Extracting frames from video:  E:\project-dataset\NV_1000.mp4
Extracting frames from video:  E:\project-dataset\NV_101.mp4
Extracting frames from video:  E:\project-dataset\NV_102.mp4
Extracting frames from video:  E:\project-dataset\NV_103.mp4
Extracting frames from video:  E:\project-dataset\NV_104.mp4
Extracting frames from video:  E:\project-dataset\NV_105.mp4
Extracting frames from video:  E:\project-dataset\NV_106.mp4
Extracting frames from video:  E:\project-dataset\NV_107.mp4
Extracting frames from video:  E:\project-dataset\NV_108.mp4
Extracting frames from video:  E:\project-dataset\NV_109.mp4
Extracting frames from video:  E:\project-dataset\NV_11.mp4
Extracting frames from video:  E:\project-dataset\NV_110.mp4
Extracting frames from video:  E:\project-dataset\NV_111.mp4
Extracting frames from vide

In [68]:
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        self.test_loss = []
        self.test_acc = []

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, batch_size=5, verbose=0)
        self.test_loss.append(loss)
        self.test_acc.append(acc)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

test_history = TestCallback((test_x, test_y))

In [58]:
data=pickle.load(open("E:\\demo_2\\hocky\\V_1\\video_summary.pkl","rb"))
output=open("E:\write_V_1.txt","w")
output.write(str(data))
output.flush()
output.close()

In [ ]:
history = model.fit_generator(
        steps_per_epoch=int(float(len_train) / float(batch_size * batch_epoch_ratio)),
        generator=train_gen,
        epochs=epochs,
        validation_data=validate_gen,
        validation_steps=int(float(len_valid) / float(batch_size)),
        callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.001, patience=patience_es, ),
                   ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=patience_lr, min_lr=1e-8, verbose=1),
                   test_history
                   ]
    )

model.save('model_1.h5')

C:\Users\VIMAL KEERTHI\AppData\Local\Temp\ipykernel_7928\6741056.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
499/499 [==============================] - ETA: 0s - loss: 0.6787 - accuracy: 0.5571
Testing loss: 0.6606144309043884, acc: 0.6675191521644592

499/499 [==============================] - 3097s 6s/step - loss: 0.6787 - accuracy: 0.5571 - val_loss: 0.6675 - val_accuracy: 0.6774 - lr: 0.0010
Epoch 2/25
414/499 [=======================>......] - ETA: 5:18 - loss: 0.6665 - accuracy: 0.6184